# Measure Graphical Structure in Bridge matrices

In [ ]:
TODO : faire un graph réellement random avec autant d'arrêtes et de noeuds (simplement mélanger les valeurs de la matrice d'adjacence) :
- quelle entropie pour erdos alors ?
- pour NBM ? Gain comparable ou significativement plus faible comparé au graph de base ?

In [1]:
import graph_tool.all as gt
from graph_tool.spectral import laplacian
import numpy as np

from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigs

import os

In [2]:
path = f"./checkpoints/"
DATASET = "laion"
UseTOPK = False
CENTER_DATASET = False
expansion_factor = 8
top_k = 20

alpha = 0.1 if UseTOPK else 0.0
beta = 0

alpha_name = str(alpha).replace('.', '')
beta_name = str(beta).replace('.', '')
bridge_name = f"{DATASET}_{"batchtopk" if UseTOPK else "MP"}_centered_{CENTER_DATASET}_{expansion_factor}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + "_bridge.npy"
null_C = f"{DATASET}_{"batchtopk" if UseTOPK else "MP"}_centered_{CENTER_DATASET}_{expansion_factor}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + "_bridge_null_C.npy"
null_D = f"{DATASET}_{"batchtopk" if UseTOPK else "MP"}_centered_{CENTER_DATASET}_{expansion_factor}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + "_bridge_null_D.npy"
null = f"{DATASET}_{"batchtopk" if UseTOPK else "MP"}_centered_{CENTER_DATASET}_{expansion_factor}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + "_bridge_null.npy"

bridge = np.load(path + bridge_name, allow_pickle=True)
null_C = np.load(path + null_C, allow_pickle=True)
null_D = np.load(path + null_D, allow_pickle=True)
null = np.load(path + null, allow_pickle=True)
energy = np.load(path + bridge_name.replace("bridge", "energy"), allow_pickle=True)

# threshold bridges with 99th percentile of null :
threshold = np.quantile(null, 0.999)
print(f"Threshold for bridge: {threshold:.2e}")
bridge[np.abs(bridge) < threshold] = 0
null_C[np.abs(null_C) < threshold] = 0
null_D[np.abs(null_D) < threshold] = 0
null[np.abs(null) < threshold] = 0

SQUARE = True
rec_types=["real-normal"]
if SQUARE:
    # square the matrix
    bridge = bridge ** 2
    null_C = null_C ** 2
    null_D = null_D ** 2
    null = null ** 2
    rec_types=["real-exponential"]

bridge_null = bridge.flatten()[np.random.permutation(len(bridge.flatten()))].reshape(bridge.shape)

print(bridge.shape)

In [3]:
# get the distribution of edge weights :
edge_weights = bridge[bridge != 0].flatten()

# sort the edge weights
edge_weights = np.sort(np.abs(edge_weights))

# cumulative distribution function :
cumulative_distribution = np.cumsum(edge_weights) / np.sum(edge_weights)
cumulative_distribution = cumulative_distribution[::-1]

import matplotlib.pyplot as plt

# Find the smallest index where the cumulative distribution is less than 0.99, 0.95, and 0.5
idx_999 = np.argmax(cumulative_distribution < 0.001)
idx_99 = np.argmax(cumulative_distribution < 0.01)
idx_95 = np.argmax(cumulative_distribution < 0.05)
idx_90 = np.argmax(cumulative_distribution < 0.1)
idx_50 = np.argmax(cumulative_distribution < 0.5)

for i in range(10):
    print(f"% of weight above 1e-{i}: {np.sum(edge_weights[edge_weights > 10**-i]) / np.sum(edge_weights)}")

p50 = edge_weights[len(edge_weights) - idx_50]
p90 = edge_weights[len(edge_weights) - idx_90]
p95 = edge_weights[len(edge_weights) - idx_95]
p99 = edge_weights[len(edge_weights) - idx_99]
p999 = edge_weights[len(edge_weights) - idx_999]
print("50th percentile:", p50)
print("90th percentile:", p90)
print("95th percentile:", p95)
print("99th percentile:", p99)
print("99.9th percentile:", p999)


print("50th percentile:", idx_50, "%:", idx_50 / len(cumulative_distribution))
print("90th percentile:", idx_90, "%:", idx_90 / len(cumulative_distribution))
print("95th percentile:", idx_95, "%:", idx_95 / len(cumulative_distribution))
print("99th percentile:", idx_99, "%:", idx_99 / len(cumulative_distribution))
print("99.9th percentile:", idx_999, "%:", idx_999 / len(cumulative_distribution))

plt.plot(cumulative_distribution[:idx_95])
plt.axvline(idx_50, color='r', linestyle='dashed', linewidth=1, label='50th percentile')
plt.axvline(idx_90, color='m', linestyle='dashed', linewidth=1, label='90th percentile')
plt.axvline(idx_95, color='g', linestyle='dashed', linewidth=1, label='95th percentile')
plt.axvline(idx_99, color='b', linestyle='dashed', linewidth=1, label='99th percentile')
plt.axvline(idx_999, color='y', linestyle='dashed', linewidth=1, label='99.9th percentile')
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('index')
plt.ylabel('cumulative distribution')
plt.show()

In [4]:
def create_graph(bridge, energy, threshold=0): # 8.27e-8): -> already thresholded using the null distribution
    A = bridge
    num_nodes = A.shape[0]

    # Build directed graph from adjacency matrix (no symmetrization)
    g = gt.Graph(directed=True)
    g.add_vertex(num_nodes)

    # Create edge weights for the graph
    weights = g.new_edge_property("double")
    for i in range(num_nodes):
        for j in range(num_nodes):
            if np.abs(A[i, j]) > threshold:
                e = g.add_edge(i, j)
                weights[e] = A[i, j]
    g.edge_properties["weight"] = weights

    # state = gt.NestedBlockState(g)

    # dS, nmoves = 0, 0
        
    # for i in range(100):
    #     ret = state.multiflip_mcmc_sweep(niter=10)
    #     dS += ret[0]
    #     nmoves += ret[1]

    # print("Change in description length:", dS)
    # print("Number of accepted vertex moves:", nmoves)
    return g
    # Run nested SBM inference on the directed graph
    state = gt.minimize_nested_blockmodel_dl(g, state_args=dict(recs=[weights], rec_types=rec_types))
    return state
    blocks = state.get_bs()[0]  # Block assignments for top-level blocks

    # Sort vertices by block and by energy within each block
    blocks = np.array(blocks)
    idx_by_block = []
    for b in np.unique(blocks):
        idxs = np.where(blocks == b)[0]
        idxs_tensor = torch.tensor(idxs)
        sorted_in_block = idxs_tensor[torch.argsort(energy[idxs_tensor], descending=True)]
        idx_by_block.append(sorted_in_block)
    
    ordered_indices = torch.cat(idx_by_block)
    sorted_bridge = bridge[ordered_indices][:, ordered_indices]
    
    return sorted_bridge, ordered_indices, blocks

g_bridge = create_graph(bridge, energy)
print(f"Graph created, with {g_bridge.num_edges()} edges and {g_bridge.num_vertices()} vertices. Average degree: {g_bridge.num_edges() / g_bridge.num_vertices()}")
g_bridge_null = create_graph(bridge_null, energy)
print(f"Graph created, with {g_bridge_null.num_edges()} edges and {g_bridge_null.num_vertices()} vertices. Average degree: {g_bridge_null.num_edges() / g_bridge_null.num_vertices()}")
g_null_C = create_graph(null_C, energy)
print(f"Graph created, with {g_null_C.num_edges()} edges and {g_null_C.num_vertices()} vertices. Average degree: {g_null_C.num_edges() / g_null_C.num_vertices()}")
g_null_D = create_graph(null_D, energy)
print(f"Graph created, with {g_null_D.num_edges()} edges and {g_null_D.num_vertices()} vertices. Average degree: {g_null_D.num_edges() / g_null_D.num_vertices()}")
g_null = create_graph(null, energy)
print(f"Graph created, with {g_null.num_edges()} edges and {g_null.num_vertices()} vertices. Average degree: {g_null.num_edges() / g_null.num_vertices()}")

In [5]:
# # create a weighted erdos-renyi model that fits the graph. Just use a SBM with 1 block :

# for g in [g_bridge, g_null_D, g_null_C, g_null]:
#     state_erdos = gt.BlockState(
#         g,
#         B=1,
#         recs=[g.ep.weight],
#         rec_types=rec_types,
#     )

#     print(f"log-likelihood of the graph: {state_erdos.entropy():.2e}")

In [5]:
def run_sbm():
    res = {}
    for g, title in [(g_bridge, "bridge"), (g_bridge_null, "bridge_null"), (g_null_D, "null_D"), (g_null_C, "null_C"), (g_null, "null")]:
        print("\n")
        state = gt.NestedBlockState(
            g,
            # B=1,
            recs=[g.ep.weight],
            rec_types=rec_types,
        )

        print(f"'Erdos' description length: {state.entropy():.2e}")

        dS, nmoves = 0, 0

        for i in range(10):
            # print("#", end="")
            ret = state.multilevel_mcmc_sweep(niter=1)
            dS += ret[0]
            nmoves += ret[1]
            print(f"entropy : {state.entropy():.2e}, S : {dS:.2e}", f"dS: {ret[0]:.2e}, nmoves: {ret[1]}, number of blocks at layer 0: {len(state.get_bs()[0])}, 1: {len(state.get_bs()[1])}, 2: {len(state.get_bs()[2])}")
        print("")
        print(f"Change in description length: {dS:.2e}")
        print(f"final description length: {state.entropy():.2e}")

        res[g] = {
            "entropy": state.entropy(),
            "dS": dS,
            "nmoves": nmoves,
            "state": state,
        }
        # print("Number of accepted vertex moves:", nmoves)

        # print("Block sizes:", state.get_bs())
    
    return res

In [6]:
states = run_sbm() # multilevel_mcmc_sweep, not B = 1

In [ ]:
states = run_sbm() # multilevel_mcmc_sweep, B = 1

In [7]:
print(state)
print(f"{state.entropy():.2e}")

In [ ]:
projected_state = state.project_level(1)

print(f"Projected state entropy: {projected_state.entropy():.2e}")

state1 = state.get_levels()[0]
print(f"Number of nodes in state 1: {state1.get_N()}")
print(state1)

block_array = state1.get_blocks()
i = 0
for v in g_bridge.vertices():
    i += 1

block_array = np.zeros(i)
for v in g_bridge.vertices():
    block_array[state1.get_blocks()[v]] += 1

nonempty_blocks = []
nonempty_idx = []
for i, b in enumerate(block_array):
    if b > 0:
        nonempty_blocks.append(b.item())
        nonempty_idx.append(i)

print("Nodes per block:", nonempty_blocks)

mat = state1.get_matrix()
print("Matrix shape:", mat.shape)
nonempty_matrix = mat[nonempty_idx][:, nonempty_idx].toarray()
#print("Matrix:", nonempty_matrix)


from matplotlib.colors import LogNorm
plt.imshow(nonempty_matrix, cmap='hot', interpolation='nearest', norm=LogNorm())
plt.colorbar()
plt.title("Matrix of block sizes")
plt.show()

In [ ]:
L = laplacian(g_bridge, weight=g_bridge.ep.weight)
eigenvalues, eigenvectors = eigs(L, k=100, which='LM', return_eigenvectors=True)
eigenvalues = np.real(eigenvalues)
eigenvectors = np.real(eigenvectors)

n_clusters = 1000
kmeans = KMeans(n_clusters=n_clusters).fit(eigenvectors)
clusters = kmeans.labels_

# # print number of nodes in each cluster
# for i in range(n_clusters):
#     print(f"Cluster {i}: {np.sum(clusters == i)} nodes")

# Plot the clusters
plt.imshow(L.toarray()[:512, :512], cmap='berlin', interpolation='nearest', vmin=-np.abs(L.toarray()).max(), vmax=np.abs(L.toarray()).max())
plt.colorbar()
plt.title('Laplacian Matrix')
plt.show()

# sort L by cluster :
perm = np.argsort(clusters)
L_sorted = L[perm][:, perm].toarray()

plt.figure(figsize=(10, 10))
plt.imshow(L_sorted[:512, :512], cmap='berlin', interpolation='nearest', vmin=-np.abs(L_sorted).max(), vmax=np.abs(L_sorted).max())
plt.colorbar()
plt.title('Laplacian Matrix sorted by clusters')
plt.show()


In [ ]:
B = g_bridge.new_vertex_property("int")
for i, cluster in enumerate(clusters):
    B[g_bridge.vertex(i)] = cluster  # Assign each vertex to its cluster

state_laplacian = gt.BlockState(
    g_bridge,
    b=B,
    recs=[g_bridge.ep.weight],
    rec_types=rec_types,
)
print(f"log-likelihood of the graph: {state_laplacian.entropy():.2e}")

In [51]:
import pickle
# save the state
with open(path + bridge_name.replace("bridge", "state"), "wb") as f:
    pickle.dump(state, f)

In [54]:
with open(path + bridge_name.replace("bridge", "state"), "rb") as f:
    loaded_state = pickle.load(f)

print(f"Loaded state log-likelihood of the graph: {loaded_state.entropy():.2e}")

In [ ]:
print(gt.collection.data.keys())

g_bridge = gt.collection.data["cond-mat"]
print(f"Graph created, with {g_bridge.num_edges()} edges and {g_bridge.num_vertices()} vertices. Average degree: {g_bridge.num_edges() / g_bridge.num_vertices()}")

state_erdos = gt.BlockState(
    g_bridge,
    B=1,
)
print(f"log-likelihood of the graph: {state_erdos.entropy():.2e}")
state = gt.NestedBlockState(
    g_bridge,
    B=1,
)
print(f"log-likelihood of the graph: {state.entropy():.2e}")
dS, nmoves = 0, 0
for i in range(100):
    ret = state.multiflip_mcmc_sweep(niter=10)
    dS += ret[0]
    nmoves += ret[1]
    print(f"entropy : {state.entropy():.2e}, S : {dS:.2e}", f"dS: {ret[0]:.2e}, nmoves: {ret[1]}, number of blocks at layer 0: {len(state.get_bs()[0])}, 1: {len(state.get_bs()[1])}, 2: {len(state.get_bs()[2])}")
print("Change in description length:", dS)
print("Number of accepted vertex moves:", nmoves)
print("Block sizes:", state)


In [65]:
state.draw()